# Test single file read using Spark

In [1]:
sc.install_pypi_package("boto3")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1625701922421_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType
import decimal
import logging
# import boto3

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# Functions


def parse_csv(line):
    """
   This function will be applied to each line read from S3 bucket's ".txt" file.

   Recall that you are working w/ comma-separated values. We want to return event object

   :param line: A line of ".txt" file in CSV format.
   :return: Common event object.
   """
    try:
        record_type_pos = 2
        record = line.split(",")
        # Filter by record_type (Trade or Quote):
        if record[record_type_pos] == "T":
            # Create event object by using values from record object and performing data type conversion
            event = [record[0], record[1], record[2], record[3], record[4], int(record[5]), record[6],
                     decimal.Decimal(record[7]), int(record[8]), None, None,
                     "T"]
                     # decimal.Decimal(record[9]),
                     # int(record[10]), "T"]  # Exists 11 values --> Should be 9.
            return event
        elif record[record_type_pos] == "Q":
            event = [record[0], record[1], record[2], record[3], record[4], int(record[5]), record[6],
                     decimal.Decimal(record[7]), int(record[8]), decimal.Decimal(record[9]),
                     int(record[10]), "Q"]
            return event
    except Exception as e:
        # Return exception as bad record and convert values to None preceding record_type == "B"
        event = [None, None, None, None, None, None, None, None, None, None, None, "B"]
        logging.error("Bad record", e)
        # print(f"Bad record: {e}")
        return event


def parse_json(line):
    """
    This function will parse through each line in the JSON formatted ".txt" file stored in S3.

    :param line: Each line of ".txt" file in JSON format.
    :return: common_event() object
    """
    try:
        record = json.loads(line)
        record_type = record["event_type"]
        # Parse records for each type and convert data type as necessary
        if record_type == "T":
            # Create event object based and
            event = [record["trade_dt"], record["file_tm"], record["event_type"], record["symbol"],
                     record["event_tm"], int(record["event_seq_nb"]), record["exchange"],
                     decimal.Decimal(record["bid_pr"]), int(record["bid_size"]), None, None, # Try place None values
                     "T"]
                     # decimal.Decimal(record["ask_pr"]),
                     # int(record["ask_size"]), "T"]
            return event
        elif record_type == "Q":
            event = [record["trade_dt"], record["file_tm"], record["event_type"], record["symbol"],
                     record["event_tm"], int(record["event_seq_nb"]), record["exchange"],
                     decimal.Decimal(record["bid_pr"]), int(record["bid_size"]), 
                     decimal.Decimal(record["ask_pr"]),
                     int(record["ask_size"]), "Q"]
            return event
    except Exception as e:
        event = [None, None, None, None, None, None, None, None, None, None, None, "B"]
        logging.error("Bad record", e)
        # print(f"Bad record: {e}")
        return event

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Define common schema -> Use for creating Spark dataframe

common_event_schema = StructType([
    StructField("trade_dt", StringType(), True),
    StructField("arrival_tm", StringType(), True),
    StructField("rec_type", StringType(), True),
    StructField("symbol", StringType(), True),
    StructField("event_tm", StringType(), True),
    StructField("event_seq_nb", IntegerType(), True),
    StructField("trade_pr", StringType(), True),
    StructField("bid_pr", DecimalType(), True),
    StructField("bid_size", IntegerType(), True),
    StructField("ask_pr", DecimalType(), True),
    StructField("ask_size", IntegerType(), True),
    StructField("partition", StringType(), True)  # THIS COLUMN IS USED FOR PARTITIONING ACROSS FILE SYSTEM
])  # Think: Why is this different from schema present in description?

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# Use the CSV Parser in Spark Transformation to Process the Source Data
spark = SparkSession.builder.master('local').appName('app').getOrCreate()  # Main entry point to Spark cluster

# # Create s3 resource object and use spark.conf.set() in order to work with S3 bucket.
# s3_resource = boto3.resource('s3')

spark.conf.set("fs.s3.endpoint", "s3-sa-east-1.amazonaws.com")

# Start Spark context manager --> used for creating RDD.
sc = spark.sparkContext  # Entry point to Spark application w/ help of Spark cluster.


# Desired output S3 path
# s3_output_path = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Parquet_Path"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Create separate ingested CSV file paths
ingested_csv_path = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/"
ingested_json_path = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_JSON_Files/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# Create Subfolder paths --> This will be used for aggregation later on.
ingest_csv_subfolder_1 = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/Ingest_CSV_Subfolder_1"
ingest_csv_subfolder_2 = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_CSV_Files/Ingest_CSV_Subfolder_2"
ingest_json_subfolder_1 = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_JSON_Files/Ingest_JSON_Subfolder_1"
ingest_json_subfolder_2 = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Ingested_JSON_Files/Ingest_JSON_Subfolder_2"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Test Read -> Note that GCFiles is our source file path

raw_csv1 = sc.textFile(
    "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/GCFiles/GC_CSV_Files/part-00000-214fff0a-f408-466c-bb15-095cd8b648dc-c000.txt")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
parsed_csv1 = raw_csv1.map(lambda line: parse_csv(line))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
parsed_csv1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

PythonRDD[26] at RDD at PythonRDD.scala:53

In [10]:
# # Test one line into parse_csv() function
# test_line = raw_csv1.take(1)
# print(type(test_line))
# parse_csv(test_line)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
spark_csv_df1 = spark.createDataFrame(parsed_csv1, common_event_schema)
spark_csv_df1

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[trade_dt: string, arrival_tm: string, rec_type: string, symbol: string, event_tm: string, event_seq_nb: int, trade_pr: string, bid_pr: decimal(10,0), bid_size: int, ask_pr: decimal(10,0), ask_size: int, partition: string]

In [26]:
spark_csv_df1.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(trade_dt='2020-08-06', arrival_tm='2020-08-06 09:30:00.0', rec_type='Q', symbol='SYMA', event_tm='2020-08-06 09:39:01.293', event_seq_nb=1, trade_pr='NYSE', bid_pr=Decimal('78'), bid_size=100, ask_pr=Decimal('78'), ask_size=100, partition='Q'), Row(trade_dt='2020-08-06', arrival_tm='2020-08-06 09:30:00.0', rec_type='Q', symbol='SYMA', event_tm='2020-08-06 09:47:20.398', event_seq_nb=2, trade_pr='NYSE', bid_pr=Decimal('77'), bid_size=100, ask_pr=Decimal('77'), ask_size=100, partition='Q'), Row(trade_dt='2020-08-06', arrival_tm='2020-08-06 09:30:00.0', rec_type='Q', symbol='SYMA', event_tm='2020-08-06 09:56:26.402', event_seq_nb=3, trade_pr='NYSE', bid_pr=Decimal('75'), bid_size=100, ask_pr=Decimal('75'), ask_size=100, partition='Q'), Row(trade_dt='2020-08-06', arrival_tm='2020-08-06 09:30:00.0', rec_type='Q', symbol='SYMA', event_tm='2020-08-06 10:03:59.522', event_seq_nb=4, trade_pr='NYSE', bid_pr=Decimal('75'), bid_size=100, ask_pr=Decimal('76'), ask_size=100, partition='Q'), Row

In [13]:
spark_csv_df1.limit(3).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------+------+--------------------+------------+--------+------+--------+------+--------+---------+
|  trade_dt|          arrival_tm|rec_type|symbol|            event_tm|event_seq_nb|trade_pr|bid_pr|bid_size|ask_pr|ask_size|partition|
+----------+--------------------+--------+------+--------------------+------------+--------+------+--------+------+--------+---------+
|2020-08-06|2020-08-06 09:30:...|       Q|  SYMA|2020-08-06 09:39:...|           1|    NYSE|    78|     100|    78|     100|        Q|
|2020-08-06|2020-08-06 09:30:...|       Q|  SYMA|2020-08-06 09:47:...|           2|    NYSE|    77|     100|    77|     100|        Q|
|2020-08-06|2020-08-06 09:30:...|       Q|  SYMA|2020-08-06 09:56:...|           3|    NYSE|    75|     100|    75|     100|        Q|
+----------+--------------------+--------+------+--------------------+------------+--------+------+--------+------+--------+---------+

In [23]:
# spark_csv_df1.repartition(3).where("partition == 'T'").limit(5).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Obtain distinct

spark_csv_df1.select("partition").distinct().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+
|partition|
+---------+
|        T|
|        Q|
+---------+

In [24]:
# Test partitioning to create separate parquet files based on "partition" column

spark_csv_df1.write.partitionBy('partition').mode('overwrite').parquet(ingest_csv_subfolder_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# Repeat Prior steps to create more parquet files based on partitioned by "partition" column

raw_csv2 = sc.textFile(
    "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/GCFiles/GC_CSV_Files/part-00000-5e4ced0a-66e2-442a-b020-347d0df4df8f-c000.txt")
parsed_csv2 = raw_csv2.map(lambda line: parse_csv(line))
spark_csv_df2 = spark.createDataFrame(parsed_csv2, common_event_schema)

spark_csv_df2.write.partitionBy('partition').mode('overwrite').parquet(ingest_csv_subfolder_2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# # Try uploading to test path instead
# test_path = "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/Test_Path"


# spark_csv_df2.write.partitionBy('partition').mode('overwrite').parquet(test_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# Use JSON Parser in Spark Transformation to Process the Source Data

raw_json1 = sc.textFile(
    "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/GCFiles/GC_JSON_Files/part-00000-092ec1db-39ab-4079-9580-f7c7b516a283-c000.txt")
parsed_json1 = raw_json1.map(lambda line: parse_json(line))
spark_json_df1 = spark.createDataFrame(parsed_json1, common_event_schema)
spark_json_df1.write.partitionBy('partition').mode('overwrite').parquet(ingest_json_subfolder_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
raw_json2 = sc.textFile(
    "s3://guidedcapstonebucket2/e-KLOB2DYT8G9BGL5F8PJ9LXKJ/GCFiles/GC_JSON_Files/part-00000-c6c48831-3d45-4887-ba5f-82060885fc6c-c000.txt")
parsed_json2 = raw_json2.map(lambda line: parse_json(line))
spark_json_df2 = spark.createDataFrame(parsed_json2, common_event_schema)
# spark_json_df2.write.partitionBy('partition').mode('overwrite').parquet(ingested_json_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
# Try uploading to test path instead

spark_json_df2.write.partitionBy('partition').mode('overwrite').parquet(ingest_json_subfolder_2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Now, let us check S3 Bucket file folders.